# Baltimore Housing Exploration


### Objective
Baltimore City has a very diverse and unequal housing situation. In this analysis I want to explore the following points:
    - Who is investing in which parts of the city?
    - What are the most expensive areas and do they overlap with crime?
    - Which parts of the city are owned by people living in their properties, which parts are investments/rentals?
    - What are the most areas where value is increasing the most?
    - What is the deal with vaccant properties? Where are they located and who owns them?
 
   
### Data 
I use open data provided by the City of Baltimore. There is not a single source of metadata and many labels lack of explanation. There is further information and links about the data sources in the README.md


First, we will import packages and load the downloaded data stored within the 'data/' folder. As we have the data stored in '.geojson' format, we will access 'features' section of the loaded json-file.  We will define some static variables.

In [1]:
import json
from collections import Counter
from area import area
import folium

In [2]:
FEAT = 'features'
PROP = 'properties'
OWNERS = 'OWNERS'
IS_VACANT = 'VACIND'
CURRENT_LAND_PRICE = 'CURRLAND'
GROUND_AREA = 'GROUND_AREA_IN_M_2'

EPSG = 4326

In [3]:
with open('data/Real_Property_Information.geojson') as f:
    d = json.load(f)
building_properties = d[FEAT]

In [29]:
# del d[FEAT]

In [31]:
CRS = d['crs']

{'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}}

Lets take a closer look at the data:
AR_OWNER = H-> principle residence of owner (exclusive owned), D -> Partially used from user, N -> not primary residence by owner 
VACIND = vaccancy indicator
ZONECODE = indicating what type of buildings are allowed in this area
MAILTOADD = mail address for this property. That can be interesting to identify bigger organisations behind owners. E.g. a company might create and LLC for each of their properties that they are holding, but they use the same mail address for each one. 

build_feats holds a list with all property features that we are interested in. 

The OWNER property is split into three parts. We will aggregate them into one string. That will help us cleaning the data and find organisations that hold multiple properties. (e.g. 'MAYOR AND CITY COUNCIL OF BALTIMORE' and 'MAYOR AND CITY COUNCIL') 

In [5]:
build_feats = ['OBJECTID', 'BLOCK', 'LOT', 'TAXBASE', 'BFCVLAND', 'BFCVIMPR', 'PERMHOME', 'CURRLAND', 'CURRIMPR',
               'USEGROUP', 'AR_OWNER', 'OWNER_ABBR', 'STDIRPRE', 'FULLADDR', 'SALEPRIC', 'NEIGHBOR', 'YEAR_BUILD',
               'LOT_SIZE', 'VACIND', 'MAILTOADD', 'ZIP_CODE']
collapse_owner = ['OWNER_1', 'OWNER_2', 'OWNER_3']

building_properties[0][PROP]

{'OBJECTID': 1,
 'PIN': '0001001',
 'PINRELATE': '0001001',
 'BLOCKLOT': '0001 001',
 'BLOCK': '0001 ',
 'LOT': '001 ',
 'WARD': '15',
 'SECTION': '370',
 'ASSESSOR': '329',
 'TAXBASE': 168600,
 'BFCVLAND': 10200,
 'BFCVIMPR': 151600,
 'LANDEXMP': 0,
 'IMPREXMP': 0,
 'CITYCRED': 0,
 'STATCRED': 0.0,
 'CCREDAMT': 0.0,
 'SCREDAMT': 0,
 'PERMHOME': 'N',
 'ASSESGRP': '2',
 'LOT_SIZE': '15-2X83-10        ',
 'NO_IMPRV': None,
 'CURRLAND': 10200,
 'CURRIMPR': 161800,
 'EXMPLAND': 0,
 'EXMPIMPR': 0,
 'FULLCASH': 0,
 'EXMPTYPE': ' ',
 'EXMPCODE': '00',
 'USEGROUP': 'C ',
 'ZONECODE': 'C-1  ',
 'SDATCODE': '11135',
 'ARTAXBAS': 165200,
 'DISTSWCH': ' ',
 'DIST_ID': ' ',
 'STATETAX': 185.02,
 'CITY_TAX': 3713.7,
 'AR_OWNER': 'N',
 'DEEDBOOK': 'MB 25915',
 'DEEDPAGE': '0340',
 'SALEDATE': '05222023',
 'OWNER_ABBR': None,
 'OWNER_1': 'NORTH AVE MARKET LLC             ',
 'OWNER_2': '                                 ',
 'OWNER_3': '                                 ',
 'FULLADDR': '2045 W NORTH AVE'

We will take a look into organisations and created a list to identify companies: 

In [6]:
company_indicators = ['LLC', 'TRUST', 'COMPANY', 'ENTERPRISE', 'INC', 'CORPORATION', 'LIMITED', 'PROPERTIES']

After a first gimplse into our dataset, lets start with cleaning the data:



In [7]:
housing_feat = []

for p in building_properties:
    building = {}
    b = p[PROP]
    owners = []
    for o in collapse_owner:
        o_raw = str(b[o]).strip()
        if o_raw != '':
            owners.append(o_raw)
    building[OWNERS] = owners
    building[GROUND_AREA] = area(p['geometry'])
    for k in build_feats:
        building[k] = b[k]
    housing_feat.append(building)

In [8]:
housing_feat[1:5]

[{'OWNERS': ['NORTH AVE MARKET LLC'],
  'GROUND_AREA_IN_M_2': 109.22893045936353,
  'OBJECTID': 2,
  'BLOCK': '0001 ',
  'LOT': '002 ',
  'TAXBASE': 49500,
  'BFCVLAND': 7000,
  'BFCVIMPR': 42500,
  'PERMHOME': 'N',
  'CURRLAND': 7000,
  'CURRIMPR': 42500,
  'USEGROUP': 'C ',
  'AR_OWNER': 'N',
  'OWNER_ABBR': None,
  'STDIRPRE': 'W',
  'FULLADDR': '2043 W NORTH AVE',
  'SALEPRIC': 250000,
  'NEIGHBOR': 'EASTERWOOD                               ',
  'YEAR_BUILD': 0,
  'LOT_SIZE': '14X83-10          ',
  'VACIND': None,
  'MAILTOADD': '1639 MORELAND AVE, 21216'},
 {'OWNERS': ['NEW YORK INC'],
  'GROUND_AREA_IN_M_2': 110.55207606655262,
  'OBJECTID': 3,
  'BLOCK': '0001 ',
  'LOT': '003 ',
  'TAXBASE': 9000,
  'BFCVLAND': 3000,
  'BFCVIMPR': 8000,
  'PERMHOME': 'N',
  'CURRLAND': 2000,
  'CURRIMPR': 7000,
  'USEGROUP': 'R ',
  'AR_OWNER': 'N',
  'OWNER_ABBR': None,
  'STDIRPRE': 'W',
  'FULLADDR': '2041 W NORTH AVE',
  'SALEPRIC': 0,
  'NEIGHBOR': 'EASTERWOOD                             

In [9]:
def clean_owners_str(list_of_owners):
    # String clean up, some whitespaces are missing
    str_owners = ' '.join(list_of_owners)
    str_owners = str_owners.replace('  ', ' ')
    str_owners = str_owners.replace('.', '')
    str_owners = str_owners.replace('&', 'AND')
    # occationally 'THE' is used for public authorities
    str_owners = str_owners.replace(', THE', '')
    str_owners = str_owners.replace('THE', '')
    return str_owners.replace('MAYOR AND CITY COUNCIL OF BALTIMORE', 'MAYOR AND CITY COUNCIL')

# 
# def convert_acers_to_meters(area_in_acers):
#     return area_in_acers * 4047
# 
# 
# def convert_sqft_to_meters(area_in_sqft):
#     return area_in_sqft * 10.764
# 
# 
# def calculate_area_from_X(value_str):
#     a_b = value_str.split('X')
#     if len(a_b) > 2:
#         return 0
#     a = a_b[0].split('-')
#     a = '.'.join(a)
#     b = a_b[1].split('-')
#     b = '.'.join(b)
#     area_in_sq_ft = float(a) * float(b)
#     return area_in_sq_ft
# 
# 
# SQ_FT = 'SQ_FT'
# ACRES = 'A'
# UNDEFINED = None
# 
# def map_units(unit_name:str) -> str:
#     UNITS = {
#         'S.F.': SQ_FT,
#         'S. F.': SQ_FT,
#         'SQ FT': SQ_FT,
#         'SF': SQ_FT,
#         'SQ. FT.': SQ_FT,
#         'SQFT': SQ_FT,
#         'SQ.FT.': SQ_FT,
#         'SF.': SQ_FT,
#         'ACRES': ACRES,
#         'ACRE': ACRES,
#         'AC': ACRES
#     }
#     unit_name = unit_name.strip()
#     return UNITS.get(unit_name, unit_name)
# 
# 
# def calculate_lot_size(lot_str):
#     lot = lot_str.upper()
#     lot = lot.replace('S.F.', 'SQ FT')
#     if 'X' in lot:
#         return calculate_area_from_X(lot)
#     lot = map_units(lot)
#     try:
#         if 'ACRES' in lot or 'ACRE' in lot:
#             lot = lot.replace('ACRES', '')
#             lot = lot.replace('ACRE', '')
#             lot = lot.replace('O', '0')
#             lot = lot.replace('-', '.')
#             lot = lot.replace(' ', '')
#             return convert_acers_to_meters(float(lot.strip()))
#         elif 'X' in lot:
#             return calculate_area_from_X(lot)
#         elif 'SQ FT' in lot or 'SQFT' in lot:
#             lot = lot.replace('SQ FT', '')
#             lot = lot.replace('SQFT', '')
#             lot = lot.replace(',', '')
#             return convert_sqft_to_meters(float(lot.strip()))
#         else:
#             print(lot)
#             return 0
#     except ValueError:
#         print('Error at: ' + lot)
#         return 0

In [10]:
def convert_vacant(vacant_str):
    if vacant_str:
        return vacant_str == 'Y'
    return None

In [11]:
owners_list = []
single_str_owners = []
for house in housing_feat:
    owner = clean_owners_str(house[OWNERS])
    single_str_owners.append(owner)
    house[OWNERS] = owner
    house['SQ_M_PRICE'] = house[CURRENT_LAND_PRICE] / house[GROUND_AREA]
    house[IS_VACANT] = convert_vacant(house[IS_VACANT])
    # house['m_2'] = calculate_lot_size(house['LOT_SIZE'])

In [12]:
sorted_owners = sorted(Counter(single_str_owners).items(), key=lambda i: i[1], reverse=True)

In [13]:
sorted_owners

[('MAYOR AND CITY COUNCIL', 10357),
 ('HOUSING AUTHORITY OF BALTIMORE CITY', 1459),
 ('ARMISTEAD HOMES CORP', 686),
 ('STATE OF MARYLAND', 510),
 ('INSULATOR DRIVE, LLC', 366),
 ('CSX TRANSPORTATION, INC', 233),
 ('BALTIMORE SFR PORTFOLIO I, LLC', 228),
 ('BALTIMORE SFR PORTFOLIO 1A, LLC', 208),
 ('BALTIMORE AFFORDABLE HOUSING DEVELOPMENT, INC', 194),
 ('MASS TRANSIT ADMINISTRATION', 168),
 ('NATIONAL RAILROAD PASSENGER CORPORATION', 154),
 ('MERRITT-HA1, LLC', 146),
 ("BELT'S LANDING, A CONDOMINIUM, INCORPORATED", 145),
 ('JOHNS HOPKINS UNIVERSITY', 144),
 ('MARYLAND HEALTH SYSTEMS, INC', 141),
 ('MADISON PARK NORTH LLC', 136),
 ('RF2, LLC', 129),
 ('BALTIMORE SFR PORTFOLIO 1B, LLC', 125),
 ('DEERA LLC', 114),
 ('DIVERSIFIED RESIDENTIAL HOMES 2, LLC', 107),
 ('FLAG HOUSE RENTAL I, LP', 102),
 ('EGBE VENTURES LLC', 95),
 ('UNIVERSITY OF MARYLAND MIDTOWN HEALTH, INC', 86),
 ('LEVY VENTURES LLC', 84),
 ('WALL STREET PROPERTIES, LLC', 83),
 ('SKYBALT HOLDINGS, LLC', 80),
 ('BALTIMORE EXCE

In [14]:
housing_lookup = {}
for h in housing_feat:
    housing_feat[h['OBJECTID']] = h

IndexError: list assignment index out of range

In [169]:
[x[IS_VACANT] for x in housing_feat[100:200]]

[False,
 True,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 True,
 True,
 True,
 None,
 None,
 None,
 None,
 True,
 None,
 None,
 None,
 False,
 True,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 True,
 True,
 True,
 True,
 True,
 None,
 None,
 True,
 None,
 True,
 True,
 True,
 None,
 True,
 None,
 False,
 None,
 True,
 True,
 True,
 True,
 True,
 None,
 None,
 True,
 None,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 None,
 None,
 None,
 None,
 None,
 True,
 True,
 True,
 True,
 None,
 True,
 True,
 None,
 True,
 True,
 None,
 None,
 None,
 None,
 True,
 False,
 True,
 None,
 None,
 True,
 None,
 None,
 None,
 None,
 True,
 None,
 None,
 None]

In [25]:
type(building_properties[0]['geometry']['coordinates'][0][0][0])

float

In [32]:
from area import area
area(building_properties[0]['geometry'])

126.5168134812962

In [28]:
building_properties[0]

{'type': 'Feature',
 'properties': {'OBJECTID': 1,
  'PIN': '0001001',
  'PINRELATE': '0001001',
  'BLOCKLOT': '0001 001',
  'BLOCK': '0001 ',
  'LOT': '001 ',
  'WARD': '15',
  'SECTION': '370',
  'ASSESSOR': '329',
  'TAXBASE': 168600,
  'BFCVLAND': 10200,
  'BFCVIMPR': 151600,
  'LANDEXMP': 0,
  'IMPREXMP': 0,
  'CITYCRED': 0,
  'STATCRED': 0.0,
  'CCREDAMT': 0.0,
  'SCREDAMT': 0,
  'PERMHOME': 'N',
  'ASSESGRP': '2',
  'LOT_SIZE': '15-2X83-10        ',
  'NO_IMPRV': None,
  'CURRLAND': 10200,
  'CURRIMPR': 161800,
  'EXMPLAND': 0,
  'EXMPIMPR': 0,
  'FULLCASH': 0,
  'EXMPTYPE': ' ',
  'EXMPCODE': '00',
  'USEGROUP': 'C ',
  'ZONECODE': 'C-1  ',
  'SDATCODE': '11135',
  'ARTAXBAS': 165200,
  'DISTSWCH': ' ',
  'DIST_ID': ' ',
  'STATETAX': 185.02,
  'CITY_TAX': 3713.7,
  'AR_OWNER': 'N',
  'DEEDBOOK': 'MB 25915',
  'DEEDPAGE': '0340',
  'SALEDATE': '05222023',
  'OWNER_ABBR': None,
  'OWNER_1': 'NORTH AVE MARKET LLC             ',
  'OWNER_2': '                                 ',
  

In [168]:
folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=state_data,
    columns=["State", "Unemployment"],
    key_on="feature.id",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Unemployment Rate (%)",
).add_to(m)

folium.LayerControl().add_to(m)

In [179]:
def save_map(data_geojson):
    m = folium.Map((39.291372, -76.599692), tiles="cartodb positron",zoom_start = 12)
    folium.GeoJson(data_geojson, name="Baltimore City Map").add_to(m)
    folium.LayerControl().add_to(m)
    m.save('map.html')

In [181]:
save_map(d)